# Cleaning and Transforming Data for F1 Machine Learning Model 

### Imports

In [35]:
import fastf1
import pandas as pd
fastf1.Cache.enable_cache("cache")

## References
- https://medium.com/@nityachintan/how-i-built-an-f1-race-prediction-app-as-my-first-machine-learning-project-7e2e9cc89826
- https://docs.fastf1.dev/


## Possible Data Requirements
- (Current Drivers only) Driver - Experience, Name, Form, Consistency
- Team - Team points trend, DNF Rate,
- Qualifying - Grid Position, Sector Times, Tyre used
- Race - Average lap time, pit strategy, stint lengths
- Context - Weather and Round Number


### Data

In [36]:
quali_2024_results = pd.read_csv("Raw_Data/2024_Qualifying_Results.csv")
quali_2024_weather = pd.read_csv("Raw_Data/2024_Qualifying_Weather.csv")

race_2024_laps = pd.read_csv("Raw_Data/2024_Race_Laps.csv")
race_2024_results = pd.read_csv("Raw_Data/2024_Race_Results.csv")
race_2024_weather = pd.read_csv("Raw_Data/2024_Race_Weather.csv")

quali_2025_results = pd.read_csv("Raw_Data/2025_Qualifying_Results.csv")
quali_2025_weather = pd.read_csv("Raw_Data/2025_Qualifying_Weather.csv")

race_2025_laps = pd.read_csv("Raw_Data/2025_Race_Laps.csv")
race_2025_results = pd.read_csv("Raw_Data/2025_Race_Results.csv")
race_2025_weather = pd.read_csv("Raw_Data/2025_Race_Weather.csv")

## Data Cleaning 

### Quali Results Data

In [37]:
quali_2024_results.isnull().sum()

DriverNumber            0
BroadcastName           0
Abbreviation            0
DriverId                0
TeamName                0
TeamColor               0
TeamId                  0
FirstName               0
LastName                0
FullName                0
HeadshotUrl            19
CountryCode             0
Position                0
ClassifiedPosition    479
GridPosition          479
Q1                      5
Q2                    121
Q3                    244
Time                  479
Status                479
Points                479
Laps                  479
RoundNumber             0
dtype: int64

In [38]:
quali_2025_results.isnull().sum()

DriverNumber            0
BroadcastName           0
Abbreviation            0
DriverId                0
TeamName                0
TeamColor               0
TeamId                  0
FirstName               0
LastName                0
FullName                0
HeadshotUrl            14
CountryCode           400
Position                0
ClassifiedPosition    400
GridPosition          400
Q1                      7
Q2                    108
Q3                    203
Time                  400
Status                400
Points                400
Laps                  400
RoundNumber             0
dtype: int64

### Dropping Null/Redundant Columns 

In [39]:
# Columns to drop from qualifying that are redundant or all null values
quali_columns_to_drop = {
    'DriverNumber', 'BroadcastName', 'Abbreviation',
       'TeamColor','FirstName', 'LastName','HeadshotUrl','Points', 
       "ClassifiedPosition", "GridPosition", "Time", "Status", "Laps"
}

quali_2024_results = quali_2024_results.drop(columns=quali_columns_to_drop)

quali_2025_results = quali_2025_results.drop(columns=quali_columns_to_drop)



### Q1, Q2 and Q3 Columns

In [40]:
# Changing dtype of the q1, 2 and 3 to represent their values 
quali_sessions = ["Q1", "Q2", "Q3"]

quali_2024_results[quali_sessions] = quali_2024_results[quali_sessions].apply(pd.to_timedelta)

quali_2025_results[quali_sessions] = quali_2025_results[quali_sessions].apply(pd.to_timedelta)


### Round Number Column

In [41]:
# Adding a prefix before the round number so can be delimited when the two datasets are joined
quali_2025_results["RoundNumber"] = "2025_" + quali_2025_results["RoundNumber"].astype("string")

quali_2024_results["RoundNumber"] = "2024_" + quali_2024_results["RoundNumber"].astype("string")



### Driver ID Column

In [42]:
quali_2025_results["DriverId"] = quali_2025_results["DriverId"].astype("string")

quali_2024_results["DriverId"] = quali_2024_results["DriverId"].astype("string")

### Country Code Column

In [43]:
# Fixing Null values in 2025 Country Codes for Qualifying by creating a map of respective country codes
country_codes = {'albon': 'THA',
        'leclerc': 'MON' ,
        'tsunoda': 'JPN' ,
        'stroll' :'CAN' ,
        'russell': 'GBR' ,
        'piastri' :'AUS' ,
        'perez' :'MEX' ,
        'norris' :'GBR' ,
        'alonso' :'ESP' ,
        'max_verstappen': 'NED' ,
        'hulkenberg' :'GER' ,
        'hamilton' :'GBR' ,
        'gasly' :'FRA' ,
        'bottas' :'FIN' ,
        'zhou' :'CHN' ,
        'ocon' :'FRA' ,
        'sainz' :'ESP' ,
        'kevin_magnussen' :'DEN', 
        'ricciardo' :'AUS' ,
        'sargeant' :'USA' ,
        'colapinto' :'ARG' ,
        'lawson': 'NZL' ,
        'bearman': 'GBR' ,
        'doohan' :'AUS',
        "bortoleto": "BRA",
        "hadjar":"FRA",
        "antonelli":"ITA" }

In [44]:
# Mapping country codes based on driver id
quali_2025_results["CountryCode"] = quali_2025_results["DriverId"].map(country_codes)

### Merging Qualifying Results Data

In [45]:
merge_quali = [quali_2024_results, quali_2025_results]

quali_data = pd.concat(merge_quali)

In [46]:
# Creating a list of all current drivers
cur_drivers = ['max_verstappen','leclerc','russell',  'sainz',
                'alonso', 'norris','piastri',
                'hamilton','hulkenberg','tsunoda', 'stroll','albon',
                'ocon',  'gasly','bearman', 'colapinto', 'lawson',
                'hadjar', 'bortoleto', 'antonelli']

# Normalizing the columns data
quali_data["DriverId"] = quali_data["DriverId"].str.strip().str.lower().astype("string")

# Filtering the df to only show data on current drivers
quali_data = quali_data[quali_data["DriverId"].isin(cur_drivers)]


### Qualifying Weather Data

### Round Number Column

In [47]:
# Adding a prefix before the round number so can be delimited when the two datasets are joined
quali_2025_weather["RoundNumber"] = "2025_" + quali_2025_weather["RoundNumber"].astype("string")

quali_2024_weather["RoundNumber"] = "2024_" + quali_2024_weather["RoundNumber"].astype("string")


### Joining Dataframes

In [ ]:
quali_weather = [quali_2024_weather, quali_2025_weather]

quali_weather_data = pd.concat(quali_weather)



### Aggregating Weather data 

In [49]:
# Aggregating all data per round number to get min, max and mean of each round 
quali_weather_aggregation = (
    quali_weather_data.groupby("RoundNumber").agg({
        "AirTemp": ["mean","min","max"],
        "Humidity": ["mean","min","max"],
        'Pressure': ["mean","min","max"],
        'WindDirection' : ["mean","min","max"],
        "WindSpeed" : ["mean","min","max"],
        "Rainfall": ["sum"]
    })
)

### Creating New Weather Dataframe

In [50]:
# Function to take the aggregated data column and create new column names
def create_new_weather_data(data, new_col_name):

    new_data = pd.DataFrame(data).reset_index()

    new_data.rename(columns={"mean":"Mean" + new_col_name, "min":"Min"+new_col_name, "max":"Max"+new_col_name},inplace=True)

    return new_data


def classify_rain(data):
    if data["sum"] == 0:
        return "No Rain"
    elif 0 < data["sum"] <= 30:
        return "Light Rain"
    elif 31 <= data["sum"] <= 60:
        return "Moderate Rain"
    else:
        return "Heavy Rain"
    

In [ ]:
# Creating new dataframes with each aggregation 
air_temp = create_new_weather_data(quali_weather_aggregation["AirTemp"], "AirTemp")
humidity = create_new_weather_data(quali_weather_aggregation["Humidity"], "Humidity")
pressure = create_new_weather_data(quali_weather_aggregation["Pressure"], "Pressure")
wind_direction = create_new_weather_data(quali_weather_aggregation["WindDirection"], "WindDirection")
wind_speed = create_new_weather_data(quali_weather_aggregation["WindSpeed"],"WindSpeed")

rainfall = pd.DataFrame(quali_weather_aggregation["Rainfall"]).reset_index()
rainfall["RainAmount"] = rainfall.apply(classify_rain,axis=1)

In [52]:
# Merging all aggregations to one df 
quali_weather_data = air_temp.merge(humidity, on="RoundNumber",how="left")
quali_weather_data = quali_weather_data.merge(pressure, on="RoundNumber",how="left")
quali_weather_data = quali_weather_data.merge(wind_direction, on="RoundNumber",how="left")
quali_weather_data = quali_weather_data.merge(wind_speed, on="RoundNumber",how="left")
quali_weather_data = quali_weather_data.merge(rainfall[["RoundNumber","RainAmount"]], on="RoundNumber",how="left")



In [ ]:
# Merging all data into one dataframe for qualifying
full_quali_data = quali_data.merge(quali_weather_data, on="RoundNumber",how="left")